In [16]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.metrics import precision_score, recall_score, accuracy_score
from sklearn.metrics import root_mean_squared_error, mean_absolute_error
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
# 1. 데이터 로딩 및 전처리

df = pd.read_excel('../../data/preprocessed_data/시도_시군구_시간대별_노인교통사고_전처리ver.xlsx', header=[0, 1])

df.columns = ['_'.join(map(str, col)).strip() for col in df.columns.values]     # 멀티인덱스를 단일 컬럼으로 변환

df_accident = df[df['기준년도_기준년도'] == '사고[건]'].reset_index(drop=True)         # '사고[건]' 행만 추출
time_columns = [col for col in df_accident.columns if '시~' in col]              # 시간대 관련 컬럼 추출

# melt로 long-form 변환
df_long = df_accident.melt(
    id_vars=['시도_시도', '시군구_시군구', '기준년도_기준년도'],
    value_vars=time_columns,
    var_name='연도_시간대',
    value_name='사고건수'
)
df_long[['연도', '시간대']] = df_long['연도_시간대'].str.split('_', expand=True)      # 연도와 시간대 분리

# 컬럼 정리
df_long = df_long[['시도_시도', '시군구_시군구', '연도', '시간대', '사고건수']].rename(
    columns={'시도_시도': '시도', '시군구_시군구': '시군구'}
)

df_long['사고건수'] = pd.to_numeric(df_long['사고건수'], errors='coerce').fillna(0).astype(int)     # 데이터 타입 변환
df_long

,시도,시군구,연도,시간대,사고건수
0,서울,합계,2020,0시~2시,174
1,서울,종로구,2020,0시~2시,6
2,서울,중구,2020,0시~2시,6
3,서울,용산구,2020,0시~2시,6
4,서울,성동구,2020,0시~2시,3
...,...,...,...,...,...
14815,경남,합천군,2024,22시~24시,0
14816,경남,창원시(통합),2024,22시~24시,20
14817,제주,합계,2024,22시~24시,13
14818,제주,제주시,2024,22시~24시,9


In [ ]:
# 2. 모델 학습 (포아송 회귀)

df_model = df_long[df_long['시군구'] != '합계'].copy()      # '합계' 데이터 제외

# 포아송 회귀 모델 생성
model = smf.glm(
    formula='사고건수 ~ C(시도) + C(시간대)',
    data=df_model,
    family=sm.families.Poisson()
).fit()

In [ ]:
# 3. 예측값 생성
df_model['예측_사고건수'] = model.predict(df_model)

In [ ]:
# 4. 실제값과 예측값 비교
comparison_df = df_model[['시도', '시간대', '사고건수', '예측_사고건수']].copy()

# 그룹별 평균값 계산
grouped_comparison = comparison_df.groupby(['시도', '시간대']).agg({
    '사고건수': 'mean',
    '예측_사고건수': 'mean'
}).reset_index()

# 결과 출력
grouped_comparison

,시도,시간대,사고건수,예측_사고건수
0,강원,0시~2시,0.555556,1.259523
1,강원,10시~12시,12.944444,12.324867
2,강원,12시~14시,13.111111,11.211026
3,강원,14시~16시,13.544444,11.724635
4,강원,16시~18시,11.244444,11.116967
...,...,...,...,...
199,충북,22시~24시,2.963636,3.979608
200,충북,2시~4시,0.872727,1.524686
201,충북,4시~6시,3.145455,4.448743
202,충북,6시~8시,9.981818,10.030156


In [ ]:
df_eval = df_model.copy()

# 예측값을 반올림하여 정수로 변환
df_eval['예측_이진'] = (df_eval['예측_사고건수'].round() >= 1).astype(int)
df_eval['실제_이진'] = (df_eval['사고건수'] >= 1).astype(int)

# 평가 지표 계산
precision = precision_score(df_eval['실제_이진'], df_eval['예측_이진'], zero_division=0)
recall = recall_score(df_eval['실제_이진'], df_eval['예측_이진'], zero_division=0)
accuracy = accuracy_score(df_eval['실제_이진'], df_eval['예측_이진'])

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Accuracy: {accuracy:.4f}")


Precision: 0.8875
Recall: 1.0000
Accuracy: 0.8875


In [ ]:
# 예측값과 실제값 배열
y_true = df_model['사고건수'].values.reshape(-1, 1)
y_pred = df_model['예측_사고건수'].values.reshape(-1, 1)

# RMSE
rmse = root_mean_squared_error(y_true, y_pred)

# MAE
mae = mean_absolute_error(y_true, y_pred)

# Cosine Similarity
cos_sim = cosine_similarity(y_true.T, y_pred.T)[0][0]

# Pearson Correlation Coefficient
correlation = np.corrcoef(y_true.flatten(), y_pred.flatten())[0, 1]

# 출력
print(f"RMSE: {rmse:.4f}")
print(f"MAE: {mae:.4f}")
print(f"Cosine Similarity: {cos_sim:.4f}")

print(f"Correlation Coefficient: {correlation:.4f}")


# ======================= 결론 ======================================

# RMS:                      11.287 -평균적으로 약 ±11건 오차
# MAE:                      7.138 - 평균 절대 오차는 약 7건 수준
# Cosine Similarity:        0.8313 - 예측값과 실제값의 패턴이 83% 이상 유사
# Correlation Coefficient:  0.6717 - 예측값과 실제값 사이에 양의 상관관계 있음

RMSE: 11.2872
MAE: 7.1376
Cosine Similarity: 0.8313
Correlation Coefficient: 0.6717
